In [1]:
# Example for calibrating eye tracking data from a Beignet recording session
# Following example in: http://aolab.wanprc.washington.edu:8888/notebooks/sijia/eyetracking_double_eye_session2337.ipynb
# Using functions in analyze/aopy/preproc.py

import os
import numpy as np
import aopy
import matplotlib.pyplot as plt

In [3]:
base_dir = "/data/preprocessed/beignet"

preprocessed_filename = 'preprocessed_te2337.hdf'
data = aopy.data.load_hdf_group(base_dir, preprocessed_filename, 'exp_data')
metadata = aopy.data.load_hdf_group(base_dir, preprocessed_filename, 'exp_metadata')


In [4]:
cursor_kinematics = data['task']['cursor'][:,[0,2,1]] # cursor (x, z, y) position on each bmi3d cycle
print(cursor_kinematics.shape)

(79523, 3)
